`To run the same project analysis, run cells 1, 6, 7`

`To generate raw files or run a new analysis, change the 'filebase' variable and run all cells`

# Import and variables

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
# import io
# import contextlib
import pandas as pd
import numpy as np
from multiprocessing import Pool
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

In [ ]:
os.chdir('validation/')

In [ ]:
validation_file = 'yeastw_opt.csv'
validation_file_out = f'{validation_file}.transitions'

typ = 'yeast'
scoretag = 'logp' # other choices: ['logp', 'logp.score>0', 'logp.score>0p1', '.scores']
tags = ['R1', 'ncbi.LL','ncbi.LR','ncbi.NL',
        'tree.LR','tree.LL','tree.NL']

outblast = 'ye288ye1573'
outmapped = f'{outblast}.mapped'

odbxref_file = 'odb10v1_gene_xrefs.tab.gz'
odbgenes_file = 'odb10v1_OG2genes.tab.gz'

ye1573_file = 'Saccharomyces_cerevisiae_yjm1573_gca_000978255.Sc_YJM1573_v1.pep.all.fa'
ye288_file = 'Saccharomyces_cerevisiae.R64-1-1.pep.all.fa'
ye288_xref_file = 'Saccharomyces_cerevisiae.R64-1-1.107.uniprot.tsv'

Change the `filebase` variable into the next cell to indicate the transitions files to test with

In [ ]:
filebase = '../../../definitivo/validation/HogProf/Eukaryota.test'
# file example: Eukaryota.test.ncbi.NL.csv.num

# Raw files download
- HogProf yeast analysis dataset
- OrthoDB orthogroup to genes
- OrthoDB gene xrefs
- Sc R64 FASTA
- Sc YJM1573 FASTA
- Sc R64 Uniprot to Ensembl

In [ ]:
%%time

!curl -O https://raw.githubusercontent.com/DessimozLab/HogProf/master/pyprofiler/notebooks/validation_set/TaSuppData/yeastw_opt.csv

In [ ]:
%%time

# fastas and xref file download
!curl -O http://ftp.ensembl.org/pub/release-106/fasta/saccharomyces_cerevisiae/pep/Saccharomyces_cerevisiae.R64-1-1.pep.all.fa.gz
!curl -O http://ftp.ensemblgenomes.org/pub/fungi/release-53/fasta/fungi_ascomycota2_collection/saccharomyces_cerevisiae_yjm1573_gca_000978255/pep/Saccharomyces_cerevisiae_yjm1573_gca_000978255.Sc_YJM1573_v1.pep.all.fa.gz
!curl -O http://ftp.ensembl.org/pub/current_tsv/saccharomyces_cerevisiae/Saccharomyces_cerevisiae.R64-1-1.107.uniprot.tsv.gz

!gunzip {ye288_file}.gz
!gunzip {ye1573_file}.gz
!gunzip {ye288_xref_file}.gz

In [ ]:
%%time

if f'{odbxref_file}.YJM1573' not in os.listdir():
    
    og2genes = pd.read_table(f'https://v101.orthodb.org/download/{odbxref_file}', header=None)
    genexref = pd.read_table(f'https://v101.orthodb.org/download/{odbgenes_file}', header=None)

    og2genes[og2genes[0].str.contains('1294385_1')].to_csv(f'{odbxref_file}.YJM1573', sep='\t', index=None, header=None)
    genexref[genexref[1].str.contains('1294385_1')].to_csv(f'{odbgenes_file}.YJM1573', sep='\t', index=None, header=None)

# BLAST
- `Saccharomyces_cerevisiae.R64-1-1` (Uniprot)
- `Saccharomyces_cerevisiae_yjm1573` (OrthoDB)

In [ ]:
%%time

# given the different strain used by orthodb, uniprot and ensembl
# mapping the ogs through a BLAST

if outblast not in os.listdir():

    !makeblastdb -in {ye1573_file} -dbtype 'prot'
    !blastp -query {ye288_file} -db {ye1573_file} -out {outblast} -outfmt=6 -max_target_seqs 6 -num_threads 32

blast = pd.read_table(outblast, header=None)[[0,1,2,10,11]]
blast.columns = ['gene1','gene2','percid','eval','bitscore']

if outmapped not in os.listdir():

    # bitscore priority with the same og ids and uniprot ids
    # allowed multiple ogs if associated to different uniprot ids

    odbxref = pd.read_table(odbxref_file+'.YJM1573', header=None)
    odbxref[1] = odbxref[1].apply(lambda x: x.split('.')[0])
    odbxref = odbxref[odbxref[2]=='NCBIproteinAcc']
    odbxrefs = dict(odbxref[[1,0]].values)

    odbgene = pd.read_table(odbgenes_file+'.YJM1573', header=None)
    odbgene = odbgene[odbgene[0].str.endswith('at2759')]

    blast = pd.merge(blast, odbxref[[0,1]].rename(
        columns={1: 'gene2', 0: 'odb2'}))
    blast = pd.merge(blast, odbgene.rename(
        columns={0: 'og2', 1: 'odb2'}))
    xref = pd.read_table(ye288_xref_file)
    blast = pd.merge(blast, xref[['xref','gene_stable_id']].rename(
        columns={'xref': 'uni1', 'gene_stable_id': 'gene1'}))

    yeast = pd.read_table(validation_file, sep=',')
    alluni = set(yeast['ProA'].tolist()+yeast['proB'].tolist())
    blast = blast[blast['uni1'].isin(alluni)]

    blast2 = blast[blast['eval']<=1e-2][blast['percid']>=70]
    b1 = blast2.sort_values('bitscore', ascending=False
                           ).drop_duplicates('uni1')

    yeast = pd.read_table(validation_file, sep=',').iloc[:,3:]
    yeast1 = pd.merge(yeast, b1[['uni1','og2']], 
                      left_on='ProA', right_on='uni1')
    yeast1 = pd.merge(yeast1, b1[['uni1','og2']], 
                      left_on='proB', right_on='uni1')
    yeast1 = yeast1.drop(['uni1_x','uni1_y'], axis=1
                        ).rename(columns={
        'og2_x': 'og1', 'og2_y': 'og2'})
    yeast1 = yeast1.drop_duplicates()
    yeast1.to_csv(outmapped, sep='\t', index=None)

# Transitions and pvalue
- `get_transitions_validation.py`
- `procedure_Orthodb_Fisher.r`

In [ ]:
%%time

# filebase = '../../../definitivo/validation/HogProf/Eukaryota.test'
def gettrans(tag):

    csv1, csv2 = f'{filebase}.{tag}.csv.num', outmapped

    # with contextlib.redirect_stdout(io.StringIO()):
    !python3 get_transitions_validation.py {csv1} {csv2} | procedure_Orthodb_Fisher.r -p 1 -pa 1 - > {os.path.basename(csv1)}.{typ}
    !python3 get_transitions_validation.py {csv1} {csv2} -c | procedure_Orthodb_Fisher.r -p 1 -pa 1 - > {os.path.basename(csv1)}.{typ}.C

# if not f'Eukaryota.test.{tags[0]}.csv.num.{typ}' in os.listdir():
Pool(9).map(gettrans, tags)

# Cotr to HogProf mapping

In [ ]:
%%time

def addnormp(yeast, tag, scoretag, penalized=True):
    ''' return a -log10pval or score based ranking dataframe given a 
    method tag and an input file containing pairs to study '''
    
    transitions_file = f'{filebase}.{tag}.csv.num.{typ}'
    if not penalized:
        transitions_file = f'{filebase}.{tag}.csv.num.{typ}.C'
    transitions = pd.read_table(transitions_file)
    transitions['ogs'] = transitions.apply(
        lambda x: '|'.join(sorted([x['G1'],x['G2']])), axis=1)

    transitions['normp'] = transitions['p'].apply(lambda x: -np.log10(x))
    transitions.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    transitions['k_score'] = transitions['k_score'].astype(float)

    ## positive scores
    if scoretag == 'logp.scores>0':
        transitions = transitions[transitions['k_score']>=0]
    
    ## negative scores --> pval == 1
    if scoretag == 'logp.scores>0p1':
        transitions['normp'] = transitions.apply(
            lambda x: x['normp'] if x['k_score']>=0 else 0)
        
    scores = dict(transitions[['ogs','normp']].values)

    ## only scores
    if scoretag == '.scores':
        transitions['k_score'] = transitions['k_score'].astype(float)
        scores = dict(transitions[['ogs','k_score']].values)

    yeast['ogs'] = yeast.apply(lambda x: '|'.join(
        sorted([x['og1'],x['og2']])), axis=1)
    yeast['normp'] = yeast['ogs'].apply(lambda x: scores.get(x))
    yeast = yeast.rename(columns={'normp': tag})

    return yeast

yeast = pd.read_table(outmapped)

yeastmapped = pd.DataFrame([addnormp(
    yeast, tag, scoretag, penalized=True).iloc[:,-1] 
                            for tag in tags]).T
yeastmapped = pd.concat([yeast, yeastmapped], axis=1)
yeastmapped.to_csv(f'{validation_file_out}.{scoretag}',
                   index=None, sep='\t')

yeastmapped = pd.DataFrame([addnormp(
    yeast, tag, scoretag, penalized=False).iloc[:,-1] 
                            for tag in tags]).T
yeastmapped = pd.concat([yeast, yeastmapped], axis=1)
yeastmapped.to_csv(f'{validation_file_out}.{scoretag}.C', 
                   index=None, sep='\t')

# Cotr AUC calculation

In [ ]:
%%time

def returnaucscores(outyeast_file, p, tags, penalizedonly=False):
    ''' calculate roc_auc for each scoring methods '''
    
    df = pd.read_table(f'{outyeast_file}')
    df = df[df['og1']!=df['og2']]
    df['id'] = df.apply(
        lambda x: '|'.join(sorted([x['ProA'],
                                   x['proB']])), axis=1)

    if not penalizedonly:
        
        df2 = pd.read_table(f'{outyeast_file}.C')
        df2 = df2[df2['og1']!=df2['og2']]
        df2['id'] = df2.apply(
            lambda x: '|'.join(sorted([x['ProA'],
                                       x['proB']])), axis=1)

    results = []
    for score in tags:

        sub = df[[score,'truth']].dropna()
        
        # for k_score type only:
        sub[score] = sub[score].astype(float)
        sub = sub.dropna()
            
        sub = sub.sort_values(score, axis=0, ascending=True)
        y_test = sub.truth
        y_pred = (sub[score]-sub[score].min()
                 )/(sub[score].max()-sub[score].min()) 
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        roc_auc = auc(fpr, tpr)

        if not penalizedonly:
            
            sub = df2[[score,'truth']].dropna()    
            
            # for k_score type only:
            sub[score] = sub[score].astype(float)
            sub = sub.dropna()
            
            sub = sub.sort_values(score, axis=0, ascending=True)
            y_test = sub.truth
            y_pred = (sub[score]-sub[score].min()
                     )/(sub[score].max()-sub[score].min()) 
            fpr2, tpr2, _ = roc_curve(y_test, y_pred)
            roc_auc2 = auc(fpr2, tpr2)
            results.append([score, roc_auc, fpr, tpr, 
                            roc_auc2, fpr2, tpr2])

        else:
            results.append([score, roc_auc, fpr, tpr])

    if not penalizedonly:
        results = pd.DataFrame(results, columns=[
            'method','rocauc','fpr','tpr',
            'rocaucp','fprp','tprp'])
    else:
        results = pd.DataFrame(results, columns=[
            'method','rocauc','fpr','tpr'])
    results = results.sort_values('rocauc', ascending=False)
    
    return df, df2, results

penalizeddf, nonpenalizeddf, respenalized = returnaucscores(
    f'{validation_file_out}.{scoretag}', 4, 
    tags, penalizedonly=False)

# penalizeddf.to_csv(f'{validation_file_out}.{scoretag}.raw', sep='\t', index=None)
# nonpenalizeddf.to_csv(f'{validation_file_out}.{scoretag}.C.raw', sep='\t', index=None)
# respenalized.to_csv(f'{validation_file_out}.{scoretag}.results', sep='\t', index=None)

# Plots

In [ ]:
df = pd.read_table(f'{validation_file_out}.{scoretag}')
df = df[df['og1']!=df['og2']]
df['id'] = df.apply(lambda x: '|'.join(
    sorted([x['ProA'],x['proB']])), axis=1)

p = 4

lens = []
for tag in tags:
    
        sign1 = len(df[df[tag]>=p][df['truth']==1]
                   )/len(df[df[tag]>=p])
        sign2 = len(df[df[tag]<p][df['truth']==1]
                   )/len(df[df[tag]<p])

        sign3 = len(df[df[tag]>=p][df['truth']==0]
                   )/len(df[df[tag]>=p])
        sign4 = len(df[df[tag]<p][df['truth']==0]
                   )/len(df[df[tag]<p])

        lens.append([
            tag, sign1, sign2, len(df[df[tag]>=p][df['truth']==1]), 
            len(df[df[tag]<p][df['truth']==1]),
            sign3, sign4, len(df[df[tag]>=p][df['truth']==0]), 
            len(df[df[tag]<p][df['truth']==0]),
                    ])

lens = pd.DataFrame(lens, columns=['method','perc_sign', 'perc_nonsign', 
                                   'len_sign', 'len_nonsign','perc_sign0',
                                   'perc_nonsign0','len_sign0','len_nonsign0'])
lens.to_csv(f'{validation_file_out}.{scoretag}.percentage', 
            sep='\t', index=None)

In [ ]:
# define a set of parameters for project uniformity
plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['legend.fontsize'] = 6

## Penalized vs non penalized

In [ ]:
df = respenalized[respenalized['method'].isin(['tree.LR'])]

fig, ax1 = plt.subplots(1, 1, figsize=(3.1,3.1), dpi=100)
ax1.plot([0, 1], [0, 1], 'k--', linewidth=1)

for i, row in enumerate(df.values.tolist()):
    ax1.plot(row[2], row[3], label=str(
        row[0]+f' {round(row[1], 3)}'), linewidth=1, alpha=1)
    ax1.plot(row[5], row[6], label=str(
        row[0]+f'.np {str(round(row[4], 3))}'), linewidth=1, alpha=1)
    
ax1.set_xlabel('False positive rate')
ax1.set_ylabel('True positive rate')

ax1.spines[['right','top']].set_visible(False)
ax1.set_title(f'{typ} PPI prediction ROC curve')
plt.legend(loc='upper left')

plt.savefig(f'{validation_file_out}.{scoretag}.results.pen.pdf')

## Tree vs NCBI

In [ ]:
df = respenalized[respenalized['method'].isin([
    'tree.LR','tree.NL','tree.LL',
    'ncbi.LR','ncbi.LL','ncbi.NL','R1'])]

fig, ax1 = plt.subplots(1, 1, figsize=(3.1,3.1), dpi=100)
ax1.plot([0, 1], [0, 1], 'k--', linewidth=1)

for i, row in enumerate(df.values.tolist()):
    ax1.plot(row[2], row[3], label=str(
        row[0]+f' {round(row[1], 3)}'), linewidth=1, alpha=1)
        
ax1.set_xlabel('False positive rate')
ax1.set_ylabel('True positive rate')

ax1.spines[['right','top']].set_visible(False)
ax1.set_title(f'{typ} PPI prediction ROC curve')
plt.legend(loc='upper left')

plt.savefig(f'{validation_file_out}.{scoretag}.results.treencbi.pdf')

## Tree LR, Jaccard hash, EPT, Binary Pearson

In [ ]:
df = pd.read_table(f'{validation_file_out}.{scoretag}')
df = df[df['og1']!=df['og2']]

results = []
for score in ['jaccard_hash','Bin_Ps','EPT_score','tree.LR']:

    sub = df[[score,'truth']].dropna()       
    sub = sub.sort_values(score, axis=0, ascending=True)
    y_test = sub.truth
    y_pred = (sub[score]-sub[score].min()
             )/(sub[score].max()-sub[score].min()) 
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    results.append([score, roc_auc, fpr, tpr])
results = pd.DataFrame(results)
results = results.sort_values(0, ascending=False)
results.to_csv(f'{validation_file_out}.{scoretag}.results.LRvJaccard', 
               sep='\t', index=None)

df = results

fig, ax1 = plt.subplots(1, 1, figsize=(3.1,3.1), dpi=100)
ax1.plot([0, 1], [0, 1], 'k--', linewidth=1)

for i, row in enumerate(df.values.tolist()):
    ax1.plot(row[2], row[3], label=
             row[0]+f' {round(row[1], 3)}', linewidth=1, alpha=1)
        
ax1.set_xlabel('False positive rate')
ax1.set_ylabel('True positive rate')

ax1.spines[['right','top']].set_visible(False)
ax1.set_title(f'{typ} PPI prediction ROC curve')
plt.legend(loc='upper left')

plt.savefig(f'{validation_file_out}.{scoretag}.results.LRvJaccard.pdf')